In [1]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

### Utilities

In [38]:
def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T

def Ad(T):
    AdT = sp.zeros(6)
    R = sp.Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def transX(a_im1):
    trA = sp.eye(4)
    trA[0,3] =  a_im1
    return trA

def transZ(d_i):
    trA = sp.eye(4)
    trA[2,3] =  d_i
    return trA

def PsFromTsd(T_sd):
    #Finner Ps fra T_sd
    #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
    #B, og derav også M, er lik som i DH
    #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
    P_d = np.array([0,0,80,1])
    P_s = T_sd@P_d

    return P_s


### Task 3-3

In [39]:
#Definerer S og M

S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,1,0,0],
              [1,0,0,890],
              [0,0,0,1]])

thetasUp = [0,0,0,0,0,0]
thetasDown = [0,0,0,0,0,0]


#Limits til roboten slik den er gitt i oppgaven. Antar at ledd 5 har limit på +-90
theta_limits = [[-180,180],[-190+90,45+90],[-120-90, 156-90],[-180,180],[-90,90],[-180,180]]

[[   0    0    0   -1    0   -1]
 [   0    1    1    0    1    0]
 [  -1    0    0    0    0    0]
 [   0 -400 -855    0 -890    0]
 [   0    0    0 -890    0 -890]
 [   0   25   25    0  445    0]]


In [40]:
#Her endres thetasGen for å teste forskjellige konfigurasjoner:
thetasGen = np.array([2,1,2,np.pi/2,np.pi/2,np.pi])
Tsd = mr.FKinSpace(M,Slist,thetasGen)
print("T_sd\n", Tsd)


T_sd
 [[-4.1198e-01 -5.8727e-02  9.0930e-01 -7.1500e+01]
 [-9.0020e-01 -1.2832e-01 -4.1615e-01  3.6009e+01]
 [ 1.4112e-01 -9.8999e-01 -9.4244e-17  5.5192e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]


In [41]:

Ps = PsFromTsd(Tsd)
print("Ps", Ps)

#theta1
thetasUp[0] = -atan2(Ps[1],Ps[0]) #minus foran fordi vinkelen er definert andre vei
thetasDown[0] = thetasUp[0]

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
Pmarked = np.array([Ps[0],Ps[1],Ps[2]-400])
b = np.sqrt((np.sqrt(Pmarked[0]**2+Pmarked[1]**2)-25)**2 + Pmarked[2]**2)#Merk: -25 pga offset i skulder

psi = np.arccos(420/a) #Vinkelen mellom den faktiske armen og den vi tegna for å få en trekant(Pga 35mm offset i elbow). Se notatbok
phi = atan2(Pmarked[2], sqrt(Pmarked[0]**2 + Pmarked[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

thetasUp[1] = np.pi/2 - (alpha + phi) #Index 1 på thetas[1] og thetas[2] gir elbow down løsning
thetasDown[1] =  np.pi/2 - (phi-alpha)

thetasUp[2] = np.pi/2 - (beta-psi)
thetasDown[2] =  -(2*np.pi - (beta+psi) - np.pi/2)

print(thetasUp, thetasDown)

#Vi har XYX euler angles. De er egentlig (-X)Y(-X) fordi det er slik S'ene er definert,.

#Elbow down:
T1 = exp6(S1, -thetasDown[0])
T2 = exp6(S2, -thetasDown[1])
T3 = exp6(S3, -thetasDown[2])

R = (T3@T2@T1@Tsd@np.linalg.inv(M)) #R er den resterende rotasjonen vi ønsker å få fra de tre siste leddene, definert i s

thetasDown[3] = -atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetasDown[4] = atan2(sqrt(1-R[0,0]**2), R[0,0])
thetasDown[5] = -atan2(R[0,1], R[0,2])


#Elbow up:
T1 = exp6(S1, -thetasUp[0])
T2 = exp6(S2, -thetasUp[1])
T3 = exp6(S3, -thetasUp[2])

R = (T3@T2@T1@Tsd@np.linalg.inv(M))

thetasUp[3] = -atan2(R[1,0], -R[2,0])
thetasUp[4] = atan2(sqrt(1-R[0,0]**2), R[0,0])
thetasUp[5] = -atan2(R[0,1], R[0,2])

Ps [  1.2436   2.7173 551.9174   1.    ]
[-1.14159265358983, 1.81578717467677 - pi, 1.3101796208137904, 0, 0, 0] [-1.14159265358983, 4.17961642332856 - pi, -4.285489810626702, 0, 0, 0]


In [46]:
#testing av analytisk løsning:

#UP
thetasUpN = np.zeros(6)
thetasUpDeg = np.zeros(6)
for i in range(0,6):
    thetasUpN[i] = N(thetasUp[i])
    thetasUpDeg[i] = np.rad2deg(thetasUpN[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

TupThetas = mr.FKinSpace(M,Slist,thetasUpN)
PUpReached = PsFromTsd(TupThetas)
#print(P_reached, P_s)
np.set_printoptions(precision=4)

#DOWN
thetasDownN = np.zeros(6)
thetasDownDeg = np.zeros(6)
for i in range(0,6):
    thetasDownN[i] = N(thetasDown[i])
    thetasDownDeg[i] = np.rad2deg(thetasDownN[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

TDownThetas = mr.FKinSpace(M,Slist,thetasDownN)
PDownReached = PsFromTsd(TDownThetas)


#fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
#R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 
thetasCalc, asd = mr.IKinSpace(Slist,M,Tsd,[0,0,0,0,0,0],0.01,0.01)

Tsd, thetasGen, TupThetas, thetasUpN, TDownThetas, thetasDownN, Ps, PUpReached, PDownReached

(array([[-4.1198e-01, -5.8727e-02,  9.0930e-01, -7.1500e+01],
        [-9.0020e-01, -1.2832e-01, -4.1615e-01,  3.6009e+01],
        [ 1.4112e-01, -9.8999e-01, -9.4244e-17,  5.5192e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([2.    , 1.    , 2.    , 1.5708, 1.5708, 3.1416]),
 array([[-4.1198e-01, -5.8727e-02,  9.0930e-01, -7.1500e+01],
        [-9.0020e-01, -1.2832e-01, -4.1615e-01,  3.6009e+01],
        [ 1.4112e-01, -9.8999e-01,  1.2068e-17,  5.5192e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-1.1416, -1.3258,  1.3102, -1.5708,  1.5708,  2.9844]),
 array([[-4.1198e-01, -5.8727e-02,  9.0930e-01, -7.1500e+01],
        [-9.0020e-01, -1.2832e-01, -4.1615e-01,  3.6009e+01],
        [ 1.4112e-01, -9.8999e-01,  6.9661e-17,  5.5192e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-1.1416,  1.038 , -4.2855, -1.5708,  1.5708, -0.2475]),
 array([  1.2436,   2.7173, 551.9174,   1.    ]),
 array([  1.24

In [48]:
Tsd, thetasUpN, thetasDownN


(array([[-4.1198e-01, -5.8727e-02,  9.0930e-01, -7.1500e+01],
        [-9.0020e-01, -1.2832e-01, -4.1615e-01,  3.6009e+01],
        [ 1.4112e-01, -9.8999e-01, -9.4244e-17,  5.5192e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-1.1416, -1.3258,  1.3102, -1.5708,  1.5708,  2.9844]),
 array([-1.1416,  1.038 , -4.2855, -1.5708,  1.5708, -0.2475]))